### **Implementacion de RAG con Mistral y LlamaIndex**. El LLM utilizado es Mistral mistral-medium-latest. Se utiliza UnstructuredPDFLoader para cargar el archivo .pdf con información de un contexto específico. Se puede responder a la pregunta de un usuario sobre ese dominio de aplicación utilizando el poder de las respuestas generativas con dicho LLM.

In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.9 

In [2]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [7]:
# Instalación de PyPDF2
!pip install PyPDF2

# Importar la biblioteca necesaria
import PyPDF2

# Ruta local del archivo PDF
local_path = "/content/00f6e6_ManualMEGAMIG500Spanish.pdf"

# Función para unir todas las páginas en un archivo .txt
def unir_paginas_en_txt(file_path):
    # Verificar que se haya proporcionado un archivo PDF
    if file_path:
        # Cargar el archivo PDF usando UnstructuredPDFLoader
        loader = UnstructuredPDFLoader(file_path=file_path)
        pdf_obj = loader.load()  # Suponiendo que esto devuelve un objeto que permite acceder a cada página

        # Crear un objeto PDF de PyPDF2 para manejar el archivo PDF
        pdf_reader = PyPDF2.PdfReader(open(file_path, "rb"))

        # Ruta de salida para el archivo .txt combinado
        output_file = "/content/combined_text.txt"

        # Abrir el archivo de texto para escribir
        with open(output_file, "w", encoding="utf-8") as txt_file:
            # Recorrer todas las páginas del PDF
            for page_num in range(len(pdf_reader.pages)): # Use len(pdf_reader.pages) instead of pdf_reader.numPages
                page = pdf_reader.pages[page_num] # Access pages using index
                page_text = page.extract_text()

                # Escribir el contenido de la página en el archivo de texto
                txt_file.write(f"Página {page_num + 1}\n\n")
                txt_file.write(page_text)
                txt_file.write("\n\n---\n\n")  # Separador entre páginas

        print(f"Se ha creado el archivo combinado: {output_file}")
    else:
        print("Sube un archivo PDF")

# Llamar a la función para unir todas las páginas en un archivo .txt
unir_paginas_en_txt(local_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Se ha creado el archivo combinado: /content/combined_text.txt


In [8]:
!pip install llama-index llama-index-llms-mistralai llama-index-embeddings-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00


In [35]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from getpass import getpass

api_key= getpass("Type your API Key")
client = MistralClient(api_key=api_key)

Type your API Key··········


In [36]:
# Función para ejecutar el modelo de Mistral
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {"role": "user", "content": user_message}
    ]
    try:
        chat_response = client.chat(
            model=model,
            messages=messages
        )
        return chat_response.choices[0].message.content
    except mistral_ai.MistralAPIException as e:
        return f"Error: {e}"

In [37]:
import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Carga la información
reader = SimpleDirectoryReader(input_files=["/content/combined_text.txt"])
documents = reader.load_data()
# Define LLM y el modelo de embedding
Settings.llm = MistralAI(model="mistral-large-latest", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)
# Create vector store index
index = VectorStoreIndex.from_documents(documents)
# Crea el motor de la query
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "¿Qué se debe hacer antes de soldar?"
)
print(str(response))

Antes de comenzar a soldar, se deben verificar varios aspectos para asegurar una operación segura y efectiva. Primero, asegúrese de que la conexión de tierra de la toma eléctrica del equipo de soldar esté conectada correctamente. También verifique que los cables de salida no estén en corto circuito y que tanto los cables de salida como los de alimentación estén en buenas condiciones, sin daños o alteraciones. Además, es importante cortar el suministro de corriente eléctrica antes de abrir la carcasa del equipo. Si el equipo presenta cualquier problema técnico, recurra al servicio técnico autorizado más cercano. Por último, realice chequeos regulares cada seis meses por parte de personal calificado luego de la instalación del equipo, incluyendo limpieza de rutina, inspección de accesorios externos y chequeo del cable de soldar.


In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
!pip install rouge-score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=261e362304a5e6da1c48707787f2a29b24966092fd2dd4a6cb5ae83d5e4005d8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [14]:
!pip install bert-score
from bert_score import score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [15]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Antes de soldar verifique el estado de las conexiones.
Revise que la conexión de tierra del enchufe esté correcta.
Utilice ropa y herramientas apropiadas para evitar dañar la vista y la piel.
Cuando se este soldando se debe usar la máscara de soldar cubriendo toda la cabeza, sólo se puede realizar la observación visual del arco eléctrico a través del visor de la máscara.
Evite la sobrecarga de su equipo revisando previamente el ciclo de trabajo de este. Tome la precaución de que la manguera de gas nunca se encuentre presionada o doblada. Siempre verifique que el cable de soldar no se encuentre maltratado.\n
Asegúrese de que no exista riesgo potencial, tanto para el operador como para la máquina, de caída de cualquier objeto extraño.
El polvo, ácido o material corrosivo en el ambiente del lugar de trabajo y medio ambiente corrosivo no deben sobrepasar los parámetros exigidos (exceptuándose los provocados a causa de la soldadura).
El equipo de soldar debe instalarse protegido del sol, la lluvia, humedad excesiva o temperaturas por bajo los -10° C o por sobre los 40° C.
Deben existir al menos 50 cms. libres alrededor del equipo de soldar para asegurar una adecuada ventilación.
No debe introducir piezas extrañas al equipo de soldar.
No deben existir vibraciones excesivas en el área alrededor del lugar de trabajo de la máquina.
Elija un área sin interferencias electromagnéticas.
Asegúrese de instalar un interruptor automático de protección en la instalación eléctrica de acuerdo a las especificaciones indicadas por el manual de servicio del equipo.
El equipo de soldar debe instalarse horizontalmente, en una superficie plana.
Si la inclinación fuera superior a 15° se le debe adicionar elementos anti vuelco para evitar la inclinación del equipo.
No se recomienda conectar su equipo de soldar a un grupo generador.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.34 seconds, 0.30 sentences/sec
ROUGE-1 score: Score(precision=0.6129032258064516, recall=0.2932098765432099, fmeasure=0.3966597077244259)
ROUGE-2 score: Score(precision=0.21428571428571427, recall=0.1021671826625387, fmeasure=0.13836477987421386)
ROUGE-L score: Score(precision=0.32903225806451614, recall=0.1574074074074074, fmeasure=0.2129436325678497)
BERTScore - Precision: 0.7237033843994141, Recall: 0.6898183822631836, F1: 0.7063547372817993


In [16]:
response = query_engine.query(
    "¿Qué ocurre si sucede Shock Eléctrico?"
)
print(str(response))

Si una persona sufre un shock eléctrico y queda inconsciente, es importante no tocarla si todavía está en contacto con algún cuerpo posiblemente energizado. Lo primero que se debe hacer es cortar el suministro eléctrico que alimenta el equipo y luego recurrir a los cuidados de primeros auxilios. Para alejar los cables y/o partes energizadas de la víctima, se recomienda utilizar trozos de madera bien seca, como una escoba o cualquier otro material aislante.


In [17]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Si la persona accidentada esta inconsciente y se sospecha un shock eléctrico, tenga la precaución de
no tocarla si ha quedado en contacto con algún cuerpo posiblemente energizado.
Corte el suministro eléctrico que alimenta el equipo y recurra a los cuidados de primeros auxilios.
Para alejar los cables y/o partes energizadas de la víctima, se recomienda utilizar trozos de madera
bien seca, como una escoba o cualquier otro material aislante.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.66 seconds, 1.52 sentences/sec
ROUGE-1 score: Score(precision=0.7875, recall=0.8289473684210527, fmeasure=0.8076923076923076)
ROUGE-2 score: Score(precision=0.6708860759493671, recall=0.7066666666666667, fmeasure=0.6883116883116883)
ROUGE-L score: Score(precision=0.75, recall=0.7894736842105263, fmeasure=0.7692307692307692)
BERTScore - Precision: 0.9254627227783203, Recall: 0.9266050457954407, F1: 0.9260335564613342


In [18]:
response = query_engine.query(
    "¿Cuáles son los requerimientos para la fuente de poder?"
)
print(str(response))

La fuente de poder, en este caso el equipo de soldar MEGAMIG 500, tiene los siguientes requerimientos:

- Voltaje de Entrada: 3PH – 380 V.
- Frecuencia: 50 Hz.
- Corriente de Entrada: 40 A.
- Potencia absorbida de Entrada: 30 KVA.
- Tensión en vacío: 20 ~ 51 V.
- Tensión de trabajo: 19,5 ~ 39 V.
- Rango de ajuste de corriente: 110 - 500 A.
- Ciclo de trabajo: 60 %.

Además, el equipo cumple con los requerimientos de la norma EN 60974-1. Para su instalación, debe ubicarse en lugares secos y con poco polvo, sin productos químicos corrosivos en los alrededores ni gases inflamables. Debe instalarse protegido del sol, la lluvia, humedad excesiva o temperaturas por debajo de los -10° C o por sobre los 40° C. También deben existir al menos 50 cms. libres alrededor del equipo para asegurar una adecuada ventilación. Si la ventilación del área de trabajo no es adecuada, se deben instalar extractores de humos.


In [19]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """El oscilograma de voltaje debe mostrar una onda de seno, la oscilación de la frecuencia no
debe exceder ± 1% del valor indicado. La oscilación del voltaje no debe superar ± 10 % del valor indicado.
El desbalance de la alimentación trifásica no debe exceder el 5%.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.78 seconds, 0.56 sentences/sec
ROUGE-1 score: Score(precision=0.15527950310559005, recall=0.5208333333333334, fmeasure=0.23923444976076555)
ROUGE-2 score: Score(precision=0.01875, recall=0.06382978723404255, fmeasure=0.02898550724637681)
ROUGE-L score: Score(precision=0.09316770186335403, recall=0.3125, fmeasure=0.14354066985645933)
BERTScore - Precision: 0.623805820941925, Recall: 0.6832370162010193, F1: 0.6521702408790588


In [20]:
response = query_engine.query(
    "¿Cuáles son las principales tareas de mantenimiento?"
)
print(str(response))

Las principales tareas de mantenimiento para el equipo de soldar Mega Mig 500 incluyen:

1. Remoción de Polvo: Asegúrate de que la máquina no esté conectada a una fuente de poder antes de proceder a la remoción de polvo.

2. Almacenamiento seguro: Cuando dejes tu equipo por un largo periodo de inactividad, hazlo en un ambiente seco, limpio y con buena ventilación.

3. Limpieza periódica: A través de personal especializado, se debe realizar limpieza por medio de aire comprimido seco para limpiar el interior del equipo de soldar. También se debe revisar si existen componentes o cables sueltos, y proceder a la reparación o reemplazo correspondiente.

4. Mantenimiento del cable de poder: Es necesario chequear periódicamente el cable de alimentación del equipo de soldar.

5. Revisión de Cables de soldar: Al momento de utilizar el equipo de soldar, el operador debe revisar que los cables de soldar no estén desconectados y no presenten cortes en el aislamiento.

6. Revisión del circuito de ga

In [21]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Remoción de Polvo .Debe asegurarse que la máquina no esté conectada a una fuente de
poder antes de proceder a la remoción de polvo.
Asegurece que al dejar su equipo por un largo periodo de inactividad, lo haga en un ambiente seco, limpio y con buena ventilación.
Limpieza periódica. A través de personal especializado, se debe realizar limpieza por medio de aire comprimido seco para limpiar el interior del equipo de soldar. También se
debe revisar si existen componentes o cables sueltos, si los hubiera, proceda a la reparación o reemplazo correspondiente. Generalmente cuando no existe polvo en demasía la limpieza
se debe realizar periódicamente, y cuando existe mucha acumulación de polvo la limpieza se debe realizar con mayor frecuencia.Mantenga el cable de poder en buen estado Es necesario chequear periódicamente el cable
de alimentación del equipo de soldar, sin embargo, es aconsejable revisar cada vez que la máquina sea utilizada de forma portátil.
Revise Cables de soldar. Al momento de utilizar el equipo de soldar, el operador debe revisar que los cables de soldar no estén desconectados, no presenten cortes en el
aislamiento, etc. En caso de que se presente alguno de estos problemas se debe proceder al cambio o reparación correspondiente.
Revise regularmente que el circuito de gas no presente fugas y se mantenga completamente sellado. Verifique ademas que el ventilador y el motor alimentador estén funcionando
correctamente sin sonidos anormales ni cables sueltos. Limpie periódicamente la boquilla de escoria y suciedad.
Utilice alambre Mig de calidad, que cuente con certificaciones. Reemplace los rodillos si se encuentran gastados o deteriorados. Apriete los rodillos contra
el alambre sin sobrepresionar para asegurar un buen desplazamiento de éste.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.14 seconds, 0.47 sentences/sec
ROUGE-1 score: Score(precision=0.8174603174603174, recall=0.6936026936026936, fmeasure=0.7504553734061931)
ROUGE-2 score: Score(precision=0.7131474103585658, recall=0.6047297297297297, fmeasure=0.6544789762340038)
ROUGE-L score: Score(precision=0.7658730158730159, recall=0.6498316498316499, fmeasure=0.7030965391621129)
BERTScore - Precision: 0.8059012293815613, Recall: 0.800682544708252, F1: 0.8032833933830261


In [22]:
response = query_engine.query(
    "¿Antes de armar o desarmar la pistola que se debe hacer?"
)
print(str(response))

Antes de armar o desarmar la pistola, es importante desconectarla de la energía eléctrica. Esto garantiza la seguridad durante el proceso.


In [23]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = "Antes de armar o desarmar la pistola se la debe desconectar de la energía eléctrica." # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.58 seconds, 1.71 sentences/sec
ROUGE-1 score: Score(precision=0.6086956521739131, recall=0.8235294117647058, fmeasure=0.7)
ROUGE-2 score: Score(precision=0.5, recall=0.6875, fmeasure=0.5789473684210527)
ROUGE-L score: Score(precision=0.5652173913043478, recall=0.7647058823529411, fmeasure=0.65)
BERTScore - Precision: 0.8530756831169128, Recall: 0.9336416125297546, F1: 0.8915421962738037


In [24]:
response = query_engine.query(
    "¿Cuándo pueden resultar peligrosas las operaciones de soldadura?"
)
print(str(response))

Las operaciones de soldadura pueden resultar peligrosas en varias situaciones. Una de ellas es cuando hay personas con marcapasos cardíacos o cualquier otro dispositivo sensible en las proximidades, ya que la inducción electromagnética producida por el equipo de soldar puede afectarlos. También es peligroso utilizar el equipo de soldar sin los paneles adecuados, ya que esto puede poner en riesgo al operador y dañar seriamente el funcionamiento del equipo. Además, si la manguera de gas está presionada o doblada, podría sufrir daños en su interior. Por otro lado, si se sobrecarga el equipo sin revisar previamente su ciclo de trabajo, se puede acortar la vida útil de los componentes y del equipo en general. Finalmente, si el cable de soldar se encuentra maltratado, también puede representar un riesgo para la seguridad.


In [25]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Si las normas de seguridad y de utilización no se observan atentamente, las operaciones de
soldadura pueden resultar peligrosas no solo para el operador, sino incluso para las personas que
se encuentren próximas al área de trabajo.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.54 seconds, 0.65 sentences/sec
ROUGE-1 score: Score(precision=0.17142857142857143, recall=0.6153846153846154, fmeasure=0.2681564245810056)
ROUGE-2 score: Score(precision=0.050359712230215826, recall=0.18421052631578946, fmeasure=0.0790960451977401)
ROUGE-L score: Score(precision=0.08571428571428572, recall=0.3076923076923077, fmeasure=0.1340782122905028)
BERTScore - Precision: 0.6921112537384033, Recall: 0.7591825127601624, F1: 0.7240971326828003


In [26]:
response = query_engine.query(
    "¿Cómo influye la velocidad del alambre?"
)
print(str(response))

La velocidad del alambre influye directamente en la corriente de soldadura. A un mismo voltaje, cuanto mayor sea la velocidad del alambre, mayor será la corriente de soldadura.


In [27]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""La velocidad del alambre influye directamente en la corriente de soldadura. A un
mismo voltaje, a mayor velocidad del alambre mayor es la corriente de soldadura.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.27 seconds, 0.79 sentences/sec
ROUGE-1 score: Score(precision=0.8571428571428571, recall=0.9230769230769231, fmeasure=0.888888888888889)
ROUGE-2 score: Score(precision=0.7407407407407407, recall=0.8, fmeasure=0.7692307692307692)
ROUGE-L score: Score(precision=0.8571428571428571, recall=0.9230769230769231, fmeasure=0.888888888888889)
BERTScore - Precision: 0.9501052498817444, Recall: 0.9623439908027649, F1: 0.9561854600906372


In [28]:
response = query_engine.query(
    "¿Cómo se debe conectar el equipo de soldar y el sistema de gas?"
)
print(str(response))

Para conectar el equipo de soldar y el sistema de gas, primero debes asegurarte de que el equipo de soldar esté correctamente conectado a la fuente de poder trifásica. Luego, conecta el cable de interconexión entre la fuente de poder y el cabezal alimentador del alambre. Asegúrate de apretar el conector en sentido reloj para garantizar una buena fijación y contacto. Además, verifica que la manguera de gas no esté presionada o doblada y que su radio de giro no sea menor a 150 mm para evitar daños en su interior. La pureza del CO2 utilizado no debe ser menor al 99,5% y su contenido de agua no debe superar el 0,005%. Siempre revisa que el cable de soldar no esté maltratado antes de comenzar a soldar.


In [29]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Asegúrese que el regulador de gas este firmemente atornillado a su conexión para evitar fugas de
gas. Luego conecte y fije firmemente el cable de calefacción a la conexión del calefactor del panel trasero
de la fuente de poder.
Conecte un extremo de la manguera de gas a la salida del regulador de presión y el otro extremo a la
entrada de gas del alimentador de alambre. """ # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.05 seconds, 0.96 sentences/sec
ROUGE-1 score: Score(precision=0.3037037037037037, recall=0.5694444444444444, fmeasure=0.3961352657004831)
ROUGE-2 score: Score(precision=0.1044776119402985, recall=0.19718309859154928, fmeasure=0.13658536585365855)
ROUGE-L score: Score(precision=0.2, recall=0.375, fmeasure=0.26086956521739135)
BERTScore - Precision: 0.711998701095581, Recall: 0.7835975289344788, F1: 0.7460842728614807


In [30]:
response = query_engine.query(
    "¿Cómo se debe realizar la instalacion y conexión del alimentador de alambre SUPER TRACK?"
)
print(str(response))

Para realizar la instalación y conexión del alimentador de alambre SUPER TRACK, primero debes seleccionar el diámetro del alambre según el trabajo a ejecutar. Luego, en función de este diámetro, elige el carrete, flexible y boquilla adecuados. Abre la cubierta del alimentador de alambre SUPER TRACK e instala el carrete en el eje correspondiente. Introduce el alambre en el tubo de entrada del mecanismo de arrastre y alínealo en la ranura del rodillo de empuje adecuado. Vuelve a centrarlo en la boquilla de salida y activa el mecanismo de presión. Finalmente, conecta la pistola en el conector correspondiente del panel frontal y aprieta firmemente. Conecta un extremo del cable de tierra al conector correspondiente en el panel frontal de la fuente de poder y atornilla firmemente. Con el otro extremo, aprisiona firmemente la pieza a soldar.


In [31]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Seleccione el diámetro del alambre según el trabajo a ejecutar. En función de este diámetro elija el
carrete, flexible y boquilla adecuados.
Abra la cubierta del alimentador de alambre SUPER TRACK e instale en el eje del carrete.
Introduzca el alambre en el tubo de entrada del mecanismo de arrastre y alínielo en la ranura del
rodillo de empuje adecuado. Vuelva a centrarlo en la boquilla de salidad y active el mecanismo de
presión. """ # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.42 seconds, 0.70 sentences/sec
ROUGE-1 score: Score(precision=0.4930555555555556, recall=0.8875, fmeasure=0.6339285714285715)
ROUGE-2 score: Score(precision=0.4125874125874126, recall=0.7468354430379747, fmeasure=0.5315315315315317)
ROUGE-L score: Score(precision=0.4791666666666667, recall=0.8625, fmeasure=0.6160714285714286)
BERTScore - Precision: 0.8349027037620544, Recall: 0.9078305959701538, F1: 0.8698408007621765


In [39]:
response = query_engine.query(
    "¿Qué sucede si el equipo de soldar es operado por largos períodos de tiempo?"
)
print(str(response))

Si el equipo de soldar es operado por largos períodos de tiempo, superando su ciclo de trabajo, se corre el riesgo de acortar la vida útil de los componentes y del equipo en general. Esto se debe a que el exceso de uso puede provocar que se active el termostato de seguridad, lo que indica un posible sobrecalentamiento o sobrecarga del equipo. Para evitar esto, es recomendable revisar previamente el ciclo de trabajo del equipo y respetar los límites establecidos.


In [40]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Cuando el equipo de soldar es operado por largos períodos de tiempo, en el panel se activará
el Led indicador de protección activada (termostato, N° 1) lo que significa que la
temperatura interna del equipo de soldar ha sobrepasado la temperatura de su diseño. En
este caso el equipo de soldar se detendrá y el ventilador seguirá funcionando. Una vez que
la temperatura baje lo suficiente, el led indicador de protección activada se apagará y el
equipo de soldar quedará en condiciones de reiniciar su operación.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.76 seconds, 0.57 sentences/sec
ROUGE-1 score: Score(precision=0.4878048780487805, recall=0.43956043956043955, fmeasure=0.4624277456647399)
ROUGE-2 score: Score(precision=0.16049382716049382, recall=0.14444444444444443, fmeasure=0.15204678362573099)
ROUGE-L score: Score(precision=0.32926829268292684, recall=0.2967032967032967, fmeasure=0.31213872832369943)
BERTScore - Precision: 0.7327222228050232, Recall: 0.7358405590057373, F1: 0.7342780828475952
